In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
# Adding system path
import sys, pathlib, os
sys.path.append(str(pathlib.Path.cwd().parent.parent))
# sys.path

In [ ]:
# Set to show warnings only once
import warnings
warnings.filterwarnings('ignore')

In [ ]:
## Setup connection to DWH
# customer = 'KIVA'
# schema = 'KIVA_PROD.OPTIML'
customer = 'OPTIML' # Use this for testing
schema = 'KIV.ACCOUNT_USAGE' # Use this for testing
username = customer + '_USERNAME'
password = customer + '_PASSWORD'
account = customer + '_ACCOUNT'

user = os.getenv(username)
password = os.getenv(password)
account = os.getenv(account)

In [ ]:
## Setup pandas
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

import numpy as np

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from tabulate import tabulate
color_scheme=["red","blue","green","orange","purple","brown","pink","gray","olive","cyan","darkviolet","goldenrod","darkgreen","chocolate","lawngreen"]

In [ ]:
# Initialize connection to Snowflake and set analysis date
from optiml.connection import SnowflakeConnConfig
# connection = SnowflakeConnConfig(accountname='jg84276.us-central1.gcp',warehousename="XSMALL_WH").create_connection()
connection = SnowflakeConnConfig(username=user,password=password,accountname=account).create_connection()

# Initialize query library
from optiml.backend.query_profile import QueryProfile
from optiml.backend.cost_profile import CostProfile, get_previous_dates
qqlib = QueryProfile(connection, schema)
cqlib = CostProfile(connection, schema)

# Initialize dates
import datetime 
# edate = datetime.date.today() - datetime.timedelta(days=1)
# sdate = edate - datetime.timedelta(days=8)
# edate = str(edate)
# sdate = str(sdate)
edate = '2022-10-12'
sdate = '2022-10-05'
print('Customer:', customer)
print('Schema:', schema)
print(str(sdate), str(edate))

In [ ]:
# Setting up autoreload for libs
%load_ext autoreload
%autoreload 2
%aimport optiml.backend.query_profile

# Query Analysis

## Failures

### Total by day

In [ ]:
df = qqlib.queries_stats_by_execution_status(sdate,edate)
# df.head()

In [ ]:
df_by_day = df.groupby(['day']).agg({'n_success': 'sum', 'n_fail': 'sum', 'credits_success': 'sum', 'credits_fail': 'sum'}).reset_index()


In [ ]:
trace1 = go.Bar(
        x = df_by_day['day'],
        y = df_by_day['n_fail'],
        name="Execution fail count",
    )


trace2  = go.Scatter(
        mode='lines+markers',
        x = df_by_day['day'],
        y = df_by_day['credits_fail'],
        name="Credits",
        yaxis='y2',
    )

data = [trace1, trace2]


layout = go.Layout(
    title_text='Query success, fail, credits per day',
    yaxis=dict(
        title="Count number",
        showgrid=False,
    ),
     yaxis2=dict(
        title="Credits", 
        overlaying="y",
        side="right",
        showgrid=False,
    ),
    xaxis=dict(
        title="Date (UTC)"
    ),
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.99
    ),
    barmode="stack"
)
fig = go.Figure(data=data, layout=layout)
fig.show()

In [ ]:
total_success = sum(df_by_day['n_success'])
total_fail = sum(df_by_day['n_fail'])
pct_fail = round(total_fail/(total_fail + total_success) * 100,2)
credits_success = round(sum(df_by_day['credits_success']),2)
credits_fail = sum(df_by_day['credits_fail'])
pct_credits_fail = round(credits_fail/(credits_fail + credits_success) * 100,2)
print('Summary Stats: Credits and counts')
print('---------------------------------')
print('Number of queries that ran to success: ', total_success)
print('Number of queries that ran to failure: ', total_fail)
print('% failed queries: ', pct_fail)
print('Credits used by queries that ran to success: ', credits_success)
print('Credits used by queries that ran to failure: ', credits_fail)
print('% credits due to failed queries: ', pct_credits_fail)
print('Credits per successful query: ', round(credits_success/total_success,4))
print('Credits per failed query: ', round(credits_fail/total_fail,4))

### By warehouse by day

In [ ]:
df_by_wh = df.groupby(["warehouse_name", "day"]).agg({'n_success': 'sum', 'n_fail': 'sum', 'credits_success': 'sum', 'credits_fail': 'sum'}).reset_index()

In [ ]:
fig = px.bar(df_by_wh, x="day", y="n_fail", color="warehouse_name", title="Number failed by warehouse")
fig.show()

In [ ]:
fig = px.bar(df_by_wh, x="day", y="credits_fail", color="warehouse_name", title="Credits failed by warehouse")
fig.show()

### By user by day

In [ ]:
df_by_user = df.groupby(["user_name", "day"]).agg({'n_success': 'sum', 'n_fail': 'sum', 'credits_success': 'sum', 'credits_fail': 'sum'}).reset_index()

In [ ]:
fig = px.bar(df_by_user, x="day", y="n_fail", color="user_name", title="Number failed by user")
fig.show()

In [ ]:
fig = px.bar(df_by_user, x="day", y="credits_fail", color="user_name", title="Credits failed by user")
fig.show()

### Top 10 expensive failing queries of past week

In [ ]:
df_expensive_queries_failed = qqlib.queries_by_execution_status(sdate,edate,'FAIL')

In [ ]:
print('Query details for top 20 expensive failing queries')
print('-------------------------------------------------')
for n in range(0,20):
    if n < len(df_expensive_queries_failed):
        # print()
        print('Query id:', df_expensive_queries_failed.iloc[n]["query_id"])
        print('User name:', df_expensive_queries_failed.iloc[n]["user_name"])
        print('Warehouse name:', df_expensive_queries_failed.iloc[n]["warehouse_name"])
        print('Query credits:', df_expensive_queries_failed.iloc[n]["credits"])
        print('Query text snippet:', df_expensive_queries_failed.iloc[n]["query_text"][0:75],'...')
        print('---------------------------------------------------------------------------------------------------')

## Expensive queries

In [ ]:
metric = 'credits'
df_v2 = qqlib.n_inefficient_queries_v2(sdate,edate,10,metric=metric)
df_v2.head()

In [ ]:
# metric = 'bytes_scanned'
# metric = 'percentage_scanned_from_cache'
# metric = 'bytes_spilled_to_local_storage'
# metric = 'bytes_spilled_to_remote_storage' -- division by 0
# metric = 'percentage_partitions_scanned' -- division by 0
# metric = 'partitions_total'
# metric = 'compilation_time_sec' #-- division by 0
# metric = 'execution_time_sec'
# metric = 'queued_provisioning_time_sec' -- division by 0
# metric = 'queued_repair_time_sec' -- division by 0
# metric = 'queued_overload_time_sec' -- division by 0
# metric = 'list_external_files_time_sec' -- division by 0
# metric = 'total_time_elapsed_sec'
metric = 'credits'
df = qqlib.n_inefficient_queries(sdate,edate,250,metric=metric)
df.head()

In [ ]:
df_unique = qqlib.get_unique_queries_with_metrics_ordered(df, metric)
df_unique.reset_index(inplace=True)
df_unique.head(20)

In [ ]:
n = 20
trace1 = go.Bar(
        x = df_unique.iloc[0:n]['query_hash'],
        y = df_unique.iloc[0:n]['n_success'],
        name="Execution success count",
    )

trace2 = go.Bar(
        x = df_unique.iloc[0:n]['query_hash'],
        y = df_unique.iloc[0:n]['n_fail'],
        name="Execution fail count",
    )

trace3  = go.Scatter(
        mode='markers+lines',
        x = df_unique.iloc[0:n]['query_hash'],
        y = df_unique.iloc[0:n][metric],
        name=metric,
        yaxis='y2',
        line=dict(color='black'),
    )

data = [trace1, trace2, trace3]

layout = go.Layout(
    title_text='Query count',
    yaxis=dict(
        title="Count number",
        showgrid=False,
    ),
     yaxis2=dict(
        title=metric, 
        overlaying="y",
        side="right",
        showgrid=False,
    ),
    xaxis=dict(
        title="Date (UTC)"
    ),
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="right",
        x=0.99
    ),
    barmode="stack"
)
fig = go.Figure(data=data, layout=layout)
fig.update_yaxes(rangemode="tozero")
fig.show()

In [ ]:
df_unique.iloc[17]

## Query execution by user per day

In [ ]:
df = qqlib.queries_by_execution_status(sdate,edate)

In [ ]:
df_by_user = df.groupby(['user_name','day']).agg({'n_success': 'sum', 'n_fail': 'sum'})
df_by_user.reset_index(inplace=True)
user_list = list(df_by_user["user_name"].unique())
# user_list

In [ ]:
# user = 'ANALYTICS_EXERCISE_USER' #significant failures
# user =  'BAMBOO' #significant failures
# user =  'DBT_DEV'
# user = 'DBT_PROD'
# user = 'FIVETRAN_USER'
# user = 'FIVETRAN_USER_DEV'
# user = 'GREGORYW' #some failures
# user = 'GREGORYW_DEV'
user = 'KIVA_API_SNAPSHOT_PROD' #all failures
# user = 'LOOKER_DEV_ADMIN' #significant failures
# user = 'LOOKER_PROD' #some failures
# user = 'LOOKER_PROD_ADMIN' #some failures
# user = 'LOOKER_RAW_DEV' #significant failures
# user = 'LOOKER_RAW_PROD' #significant failures
# user = 'MAXH_DEV' #some failures
# user = 'ML_SERVICE_DEV'
# user = 'ML_SERVICE_PROD'
# user = 'PATRICKL'
# user = 'PATT' #significant failures
# user = 'ROBS' #significant failures
# user = 'TEST_EXERCISE_AS' #some failures
# user = 'TEST_EXERCISE_CM' #some failures
# user = 'TEST_EXERCISE_JAR' #some failures
# user = 'TEST_EXERCISE_LAZ' #some failures
# user = 'TEST_EXERCISE_RDN' #some failures
# user = 'VERTEX_API_DEV'
# user = 'VERTEX_API_DEV_JENKINS'
# user = 'VERTEX_API_PROD'
# user = 'WORKSHEETS_APP_USER'
df_user = df_by_user[df_by_user["user_name"] == user]
df_user.reset_index(drop=True,inplace=True)

In [ ]:
trace1 = go.Bar(
        x = df_user['day'],
        y = df_user['n_success'],
        name="Execution success count",
    )

trace2 = go.Bar(
        x = df_user['day'],
        y = df_user['n_fail'],
        name="Execution fail count",
    )

data = [trace1, trace2]

layout = go.Layout(
    title_text='Query count',
    yaxis=dict(
        title="Count number",
        showgrid=False,
    ),
     yaxis2=dict(
        title="Number of times ran", 
        overlaying="y",
        side="right",
        showgrid=False,
    ),
    xaxis=dict(
        title="Date (UTC)"
    ),
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.99
    ),
    barmode="stack"
)
fig = go.Figure(data=data, layout=layout)
fig.show()

In [ ]:
## all failures for a user between start and end date
df=qqlib.get_queries(start_date=sdate,end_date=edate,user=user,es="FAIL", n=1500)
df_unique = qqlib.get_unique_queries(df)
df_unique.sort_values('execution_status', inplace=True, ascending=False)
df_unique.reset_index(inplace=True)
df_unique.head()

## Query execution by warehouse per day

In [ ]:
df = qqlib.queries_by_execution_status(sdate,edate)

In [ ]:
df_by_wh = df.groupby(['warehouse_name','day']).agg({'n_success': 'sum', 'n_fail': 'sum'})
df_by_wh.reset_index(inplace=True)
wh_list = list(df_by_wh["warehouse_name"].unique())
# wh_list

In [ ]:
# wh = 'DAILY_REFRESH_WH'
# wh = 'DEV_WH'
# wh = 'ML_WH'
# wh ='PROD_WH'
wh = 'Unassigned'
df_wh = df_by_wh[df_by_wh["warehouse_name"] == wh]
df_wh.reset_index(drop=True,inplace=True)

In [ ]:
trace1 = go.Bar(
        x = df_wh['day'],
        y = df_wh['n_success'],
        name="Execution success count",
    )

trace2 = go.Bar(
        x = df_wh['day'],
        y = df_wh['n_fail'],
        name="Execution fail count",
    )

data = [trace1, trace2]

layout = go.Layout(
    title_text='Query count',
    yaxis=dict(
        title="Count number",
        showgrid=False,
    ),
     yaxis2=dict(
        title="Number of times ran", 
        overlaying="y",
        side="right",
        showgrid=False,
    ),
    xaxis=dict(
        title="Date (UTC)"
    ),
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.99
    ),
    barmode="stack"
)
fig = go.Figure(data=data, layout=layout)
fig.show()

## Get n queries given start_date, and end_date, or user or warehouse, execution_status

In [ ]:
sdate = '2022-10-11'
edate = '2022-10-12'
es = 'FAIL'
n = 1500
wh = 'Unassigned'
df = qqlib.get_queries(start_date=sdate,end_date=edate,es=es, n=n)
df = df.fillna('Unassigned')
df = df[df['warehouse_name']==wh]
df_unique = qqlib.get_unique_queries(df)
df_fails = df_unique.sort_values('execution_status', ascending=False).reset_index()
df_fails.head()

## Get most inefficient query by metric - credits, total time elapsed etc. as query_id and details

In [ ]:
# metric = 'bytes_scanned'
# metric = 'percentage_scanned_from_cache'
# metric = 'bytes_spilled_to_local_storage'
# metric = 'bytes_spilled_to_remote_storage' -- division by 0
# metric = 'percentage_partitions_scanned' -- division by 0
# metric = 'partitions_total'
# metric = 'compilation_time_sec' #-- division by 0
# metric = 'execution_time_sec'
# metric = 'queued_provisioning_time_sec' -- division by 0
# metric = 'queued_repair_time_sec' -- division by 0
# metric = 'queued_overload_time_sec' -- division by 0
# metric = 'list_external_files_time_sec' -- division by 0
# metric = 'total_time_elapsed_sec'
metric = 'credits'
sdate = "2022-08-12"
edate = "2022-10-12"
df = qqlib.n_inefficient_queries(sdate,edate,250,metric=metric)
df.head()

## Get most inefficient query by metric - credits, total time elapsed etc. with details and grouped by query text

In [ ]:
# metric = 'bytes_scanned'
# metric = 'percentage_scanned_from_cache'
# metric = 'bytes_spilled_to_local_storage'
# metric = 'bytes_spilled_to_remote_storage' -- division by 0
# metric = 'percentage_partitions_scanned' -- division by 0
# metric = 'partitions_total'
# metric = 'compilation_time_sec' #-- division by 0
# metric = 'execution_time_sec'
# metric = 'queued_provisioning_time_sec' -- division by 0
# metric = 'queued_repair_time_sec' -- division by 0
# metric = 'queued_overload_time_sec' -- division by 0
# metric = 'list_external_files_time_sec' -- division by 0
# metric = 'total_time_elapsed_sec'
metric = 'credits'
df = qqlib.n_inefficient_queries(sdate,edate,250,metric=metric)
df.head()

In [ ]:
df_unique = qqlib.get_unique_queries_with_metrics_ordered(df, metric)
df_unique.reset_index(inplace=True)
df_unique.head()

In [ ]:
n = 20
trace1 = go.Bar(
        x = df_unique.iloc[0:n]['query_hash'],
        y = df_unique.iloc[0:n]['n_success'],
        name="Execution success count",
    )

trace2 = go.Bar(
        x = df_unique.iloc[0:n]['query_hash'],
        y = df_unique.iloc[0:n]['n_fail'],
        name="Execution fail count",
    )

trace3  = go.Scatter(
        mode='markers+lines',
        x = df_unique.iloc[0:n]['query_hash'],
        y = df_unique.iloc[0:n][metric],
        name=metric,
        yaxis='y2',
        line=dict(color='black'),
    )

data = [trace1, trace2, trace3]

layout = go.Layout(
    title_text='Query count',
    yaxis=dict(
        title="Count number",
        showgrid=False,
    ),
     yaxis2=dict(
        title=metric, 
        overlaying="y",
        side="right",
        showgrid=False,
    ),
    xaxis=dict(
        title="Date (UTC)"
    ),
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="right",
        x=0.99
    ),
    barmode="stack"
)
fig = go.Figure(data=data, layout=layout)
fig.update_yaxes(rangemode="tozero")
fig.show()

### Warning note

<div class="alert alert-danger">
    
* We do not recommend the naive approach since it can have very limited ROI

</div>

In [ ]:
##TODO: Temporal distribution of expensive queries by warehouse
df_hash = qqlib.get_unique_query(df)
# df_hash.head()

In [ ]:
df_by_execution_status = df_hash.groupby(["execution_status"])["execution_success","execution_fail","execution_count","credits"].sum().reset_index()

# Pie charts for expensive queries by execution status
fig = make_subplots(
    rows=1, cols=2,
    specs=[[{"type": "pie"},{"type": "pie"}]],
    subplot_titles=("Count", "Credits")
)

fig.add_trace(go.Pie(labels=df_by_execution_status['execution_status'].tolist(), values=df_by_execution_status['execution_count'].tolist(), 
                     name="Execution Count", rotation=45, marker_colors=["red","green"]),row=1,col=1)
fig.add_trace(go.Pie(labels=df_by_execution_status['execution_status'].tolist(), values=df_by_execution_status['credits'].tolist(),
                     name='Credits', rotation=45, marker_colors=["red","green"]),row=1,col=2)

fig.update_layout(
    title={
        'text': "Expensive queries breakdown by execution status",
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
fig.show()

In [ ]:
df_by_user = df_hash.groupby(["user_name"])["execution_success","execution_fail","execution_count","credits"].sum().reset_index()
print('Credit and count of expensive queries by user')
print('---------------------------------------------')
print(tabulate(df_by_user, headers='keys', tablefmt='rounded_outline', showindex=False))


In [ ]:
# Pie charts for expensive queries by execution status
fig = make_subplots(
    rows=1, cols=2,
    specs=[[{"type": "pie"},{"type": "pie"}]],
    subplot_titles=("Count", "Credits")
)

fig.add_trace(go.Pie(labels=df_by_user['user_name'].tolist(), values=df_by_user['execution_count'].tolist(), 
                     name="Execution Count", rotation=45, marker_colors=color_scheme),row=1,col=1)
fig.add_trace(go.Pie(labels=df_by_user['user_name'].tolist(), values=df_by_user['credits'].tolist(),
                     name='Credits', rotation=45, marker_colors=color_scheme),row=1,col=2)

fig.update_layout(
    title={
        'text': "Expensive queries breakdown by user",
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
fig.show()

In [ ]:
df_by_wh = df_hash.groupby(["warehouse_name"])["execution_success","execution_fail","execution_count","credits"].sum().reset_index()
print('Credit and count of expensive queries by user')
print('---------------------------------------------')
print(tabulate(df_by_wh, headers='keys', tablefmt='rounded_outline', showindex=False))

In [ ]:
# Pie charts for expensive queries by execution status
fig = make_subplots(
    rows=1, cols=2,
    specs=[[{"type": "pie"},{"type": "pie"}]],
    subplot_titles=("Count", "Credits")
)

fig.add_trace(go.Pie(labels=df_by_wh['warehouse_name'].tolist(), values=df_by_user['execution_count'].tolist(), 
                     name="Execution Count", rotation=45, marker_colors=color_scheme),row=1,col=1)
fig.add_trace(go.Pie(labels=df_by_wh['warehouse_name'].tolist(), values=df_by_user['credits'].tolist(),
                     name='Credits', rotation=45, marker_colors=color_scheme),row=1,col=2)

fig.update_layout(
    title={
        'text': "Expensive queries breakdown by warehouse",
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
fig.show()

In [ ]:
## Most expensive queries
df_by_hash = df_hash.groupby(["query_hash"])["execution_success","execution_fail","credits"].sum().reset_index().sort_values('credits', ascending=False)
trace1 = go.Bar(
        x = df_by_hash['query_hash'],
        y = df_by_hash['execution_success'],
        name="Execution success count",
        yaxis='y2',
        marker=dict(color='green'),
        opacity=0.5
    )

trace2 = go.Bar(
        x = df_by_hash['query_hash'],
        y = df_by_hash['execution_fail'],
        name="Execution fail count",
        yaxis='y2',
        marker=dict(color='red'),
        opacity=0.5
    )

trace3  = go.Scatter(
        mode='lines+markers',
        x = df_by_hash['query_hash'],
        y = df_by_hash['credits'],
        name="Credits",
        line=dict(color='black'),
    )


data = [trace1, trace2, trace3]

layout = go.Layout(
    title_text='Most expensive queries',
    yaxis=dict(
        title="Total Credits",
        showgrid=False,
        range=[0, math.ceil(max(df_by_hash["credits"]))+10]
    ),
    yaxis2=dict(
        title="Number of times ran", overlaying="y",
        side="right",
#         position=0.98,
        showgrid=False,
        range=[0, math.ceil(max(df_by_hash["execution_success"] + df_by_hash["execution_fail"]))+10]
    ),
    xaxis=dict(
        title="Query Hash"
    ),
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="right",
        x=0.99
    ),
    barmode="stack"
)
fig = go.Figure(data=data, layout=layout)
fig.show()

In [ ]:
df_by_hash = df_hash.groupby(["query_hash"])["execution_success","execution_fail","execution_count","credits"].sum().reset_index()
df_by_hash = df_by_hash.sort_values('credits', ascending=False)
print(tabulate(df_by_hash, headers='keys', tablefmt='rounded_outline', showindex=False))

In [ ]:
## Check times of 5 most expensive queries with WH credit usage peaks
# q1 = df_hash[df_hash['query_hash'] == '6ef334ed61427031d52acf53a055ab57'].sort_values('start_time')
# q1[["warehouse_name","start_time","end_time"]]
q2 = df_hash[df_hash['query_hash'] == '295230eae35c6270a0e240fca4a0be6f'].sort_values('start_time')
q2[["query_id", "warehouse_name","start_time","end_time"]]
# q3 = df_hash[df_hash['query_hash'] == '233655c56c9344d5c303bdd8f4b9f264'].sort_values('start_time')
# q3[["warehouse_name","start_time","end_time"]]
# q4 = df_hash[df_hash['query_hash'] == 'd154d97ba4dbd4fd677baa48d6b47b71'].sort_values('start_time')
# q4[["warehouse_name","start_time","end_time"]]
# q5 = df_hash[df_hash['query_hash'] == '5cb178243f7dc6005e8141a72f9bc895'].sort_values('start_time')
# q5[["warehouse_name","start_time","end_time"]]
# q6 = df_hash[df_hash['query_hash'] == '535f3564c40d364959615108ceee4ea0'].sort_values('start_time')
# q6[["warehouse_name","start_time","end_time"]]

### Weekly change of 10 top expensive queries

In [ ]:
def get_n_expensive_query_hashes(sdate, edate, n):
    df_expensive = qqlib.n_expensive_queries(sdate, edate, 200)
    df_hash = get_unique_query(df_expensive).sort_values('credits', ascending=False)
    hash_list = df_hash["query_hash"].unique()
    return hash_list[0:n]

In [ ]:
import pandas as pd 
sdate = '2022-01-01'
n_periods = 9
frequency = '1M'
sdates = pd.date_range(sdate, periods=n_periods, freq=frequency)
edates = pd.date_range(sdates[1], periods=n_periods, freq=frequency)
start_dates = [d.strftime('%Y-%m-%d') for d in sdates]
end_dates = [d.strftime('%Y-%m-%d') for d in edates]
df_hash = pd.DataFrame()

for idx, s in enumerate(start_dates):
    df_hash[s] = pd.DataFrame(get_n_expensive_query_hashes(start_dates[idx], end_dates[idx], 10), columns=[start_dates[0]])


df_hash

In [ ]:
cols = df_hash.columns.values
repeat = []
for idx, col in enumerate(cols[:-1]):
    repeat.append(sum(el in df_hash[cols[idx]].to_list() for el in df_hash[cols[idx+1]].to_list()))

print(repeat)

In [ ]:
## 14 days
from statistics import mean, median
mean(repeat), median(repeat)

### Analysis

<div class="alert alert-info">
    
* Naively optimizing the most expensive query is not an effective strategy. Some queries:
    * Fail to run to completion generating no ROI but use resources
    * Run more frequently than others so optimizing them will lead to better ROI
* 5% of queries fail to run to completion by count but they consume ~ 6% of credits
* Majority of expensive queries come from DEV_WH - which makes sense, but larger fraction comes from DAILY_REFRESH_WH by credits than by count which might have optimization opportunity.
* DBT_DEV, DBT_PROD write > 90% of the top 200 most expensive queries. This is inspite of the fact that their total credit consumption is <30% and much less than that of the most expensive user i.e. FIVETRAN_USER. There may be opportunity to do DBT specific optimization for Snowflake.
* Manually checking 5 most expensive queries showed 4 of them are consistent with peaks in warehouse usage:
    * Query Hash: 6ef334ed61427031d52acf53a055ab57 (DAILY_REFRESH_WH, 2 pm), 295230eae35c6270a0e240fca4a0be6f (DEV_WH, 12 pm), 233655c56c9344d5c303bdd8f4b9f264 (DAILY_REFRESH_WH, 9 am), d154d97ba4dbd4fd677baa48d6b47b71 (DAILY_REFRESH_WH, 2:50 pm), 5cb178243f7dc6005e8141a72f9bc895 (DEV_WH, 1 pm)
                                                                                                                                               
* Queries aggregated by query hash (i.e. query text matching exactly) show:                                                                                                 
    * Query IDs: 01a78358-0604-ec7d-0000-08d15dc9f6fe, 01a78350-0604-ec7d-0000-08d15dc9f016  might be responsible for increase in compute consumption for DEV_WH on 8th Oct. (alternatively could be WH scaling/size change as well)
    * Query Hash: 6ef334ed61427031d52acf53a055ab57 cumulatively contributes to the most number of credits (~28) and runs to completion 12 times 
    * Query Hash: 49539ab6df6b8a1df712dbbd70efe3e3 fails 3 times it runs and costs ~3 credits
    * Multiple queries that fail once during the analysis period

</div>

### Actions and Recommendations

<div class="alert alert-success">

* ##TODO:
    * Are these queries all current and serving a business purpose - if not remove them
    * Look at other queries that feed reports that no one is looking at and remove them
    * Optimize queries that fail to run or cumulatively consume more credits than the queries that consume the most credits but run fewer time
    * For highest ROI generate query hashes using templates so that queries can be grouped irrespective of parameter values
* For queries with higher spillage consider a larger instance
* For queries that scan a large number of partitions consider:
    * Search Optimization enable for selective queries
    * Autoclustering enable with thoughtfully chosen cluster keys
* SCD2 for update queries (https://community.snowflake.com/s/article/Building-a-Type-2-Slowly-Changing-Dimension-in-Snowflake-Using-Streams-and-Tasks-Part-1)
    
</div>

In [ ]:
df_hash_merged = pd.merge(df_by_hash, df_hash[['query_hash', 'query_text','user_name','warehouse_name']], on=["query_hash"])
df_hash_merged = df_hash_merged.drop_duplicates(subset=['query_hash']).reset_index(drop=True)

In [ ]:
df.head(10)

## Search for queries by any column

### Query text

In [ ]:
update_iterable = df[df['query_text'].str.contains("merge into ", case=False)]["query_text"].reset_index(drop=True)
print(update_iterable.iloc[0])

### Start and end time

### Query text

#### Query 1

In [ ]:
print(df_hash_merged.iloc[0]["query_text"])

#### Query 2

In [ ]:
print(df_hash_merged.iloc[1]["query_text"])

#### Query 3

In [ ]:
print(df_hash_merged.iloc[12]["query_text"])

## Queries that spill to storage

In [ ]:
df=qqlib.n_queries_spill_to_storage(sdate,edate,5)
df.head()

In [ ]:
## Most expensive queries

trace1  = go.Scatter(
        mode='lines+markers',
        x = df['query_id'],
        y = df['bytes_spilled_to_remote_storage'],
        name="Bytes Spilled Remote",
        marker_color='crimson'
    )

trace2  = go.Scatter(
        mode='lines+markers',
        x = df['query_id'],
        y = df['bytes_spilled_to_local_storage'],
        name="Bytes Spilled Local",
        marker_color='purple'
    )


data = [trace1, trace2]


layout = go.Layout(
    title_text='Queries that spilled the most to storage',
    yaxis=dict(
        # range = [0, 100],
        side = 'left',
        title="Bytes spilled"
        
    ),
    xaxis=dict(
        title="Query ID"

    )
)
fig = go.Figure(data=data, layout=layout)
fig.show()

### Query text

#### Query 1

In [ ]:
print(df.iloc[0]["query_text"])

#### Query 2

In [ ]:
print(df.iloc[1]["query_text"])

#### Query 3

In [ ]:
print(df.iloc[2]["query_text"])

### ANALYSIS

<div class="alert alert-info">
##TODO: @Saravana sum local and remote query spillage and plot a graph against warehouse to see which warehouse should be upgraded.
    
 Queries 1,2 and 3 fail after execution time for each being 29 minutes.
* Query 1 (query_id: 01a78b77-0604-f44d-0000-08d15ddd870e) - 300 GB spilling to local storage and 200 GB to remote storage.
* Query 2 (query_id: 01a78b55-0604-f44e-0000-08d15ddd55da) - 170 GB splilling to local storge and 90 GB to remote.
* Query 3 (query_id: 01a78b55-0604-f44e-0000-08d15ddd55d6) - 245 GB spilling to local and 78 GB to remote storage.

Most partitions are scanned in the table for these 3 queries.
There are also multiple join and select statements for each query.
</div>




### Optiml TODO



<div class="alert alert-warning">
    
* Check if best virtual warehouse practices can be applied to most queries in this warehouse
</div>

### Optiml Recommendation




<div class="alert alert-success">

* Utilize a larger warehouse - Warehouse sizes of Xsmall and Small are used. 100's of GB are spilled due to insufficient resources. 

* Optimize the query - Reducing the number of JOINS, SELECT and UNION statements would improve the performance.

* Reduce the data that is being processed (Ex: Redunant columns used in computation).

* Split processing into multiple steps.

## Queries that scanned the most data

In [ ]:
df=qqlib.n_queries_scanned_most_data(sdate,edate,5)
df.head() 

In [ ]:
##TODO: Put labels on the axis
## Queries that scanned most data

trace1  = go.Scatter(
        mode='lines+markers',
        x = df['query_id'],
        y = df['partitions_scanned'],
        name="Partitions Scanned",
        marker_color='crimson'
    )

data = [trace1]

layout = go.Layout(
    title_text='Queries that scanned the most partitions',
    yaxis=dict(
        # range = [0, 100],
        side = 'left',
        title="Bytes spilled"
        
    ),
    xaxis=dict(
        title="Query ID"

    )
    
)
fig = go.Figure(data=data, layout=layout)
fig.show()

### Query text

#### Query 1

In [ ]:
print(df.iloc[0]["query_text"])

### ANALYSIS

* Inadequte pruning is observed. 99% of the partitions on the table are scanned.
* Execution time is 3.5 minutes.
* 2 GB is spilled onto local storage

#### Query 2

In [ ]:
print(df.iloc[1]["query_text"])

### ANALYSIS

* Inadequte pruning is observed. 99% of the partitions on the table are scanned.
* Execution time is 7.7 minutes.
* 16.9 GB is spilled onto local storage.

#### Query 3

In [ ]:
print(df.iloc[2]["query_text"])

### ANALYSIS

* Inadequte pruning is observed. 99% of the partitions on the table are scanned.
* Execution time is 3.5 minutes.
* 2 GB is spilled onto local storage.

### OPTIML Recommendations

##TODO: @saravana Look at the wording on the recommendations and update them approrpriately - create cluster keys on time stamps
* Queries are the same.
* Cluster keys can be defined to reduce partitions scanned. 
* Choose cluster key that appears frequently in a WHERE clause
* Scaling up warehouse would reduce bytes splilling to local storage
* Query can be optimized by reducing number of JOIN statements, eliminating redundant SELECT statements and using DISTINCT clauses.

## Most cached queries

In [ ]:
df=qqlib.n_most_cached_queries(sdate,edate,5)
df.head() 


In [ ]:
## Queries that scanned the most from cache

trace1  = go.Scatter(
        mode='lines+markers',
        x = df['query_id'],
        y = df['percent_scanned_from_cache'],
        name="Percent Scanned From Cache",
        marker_color='crimson'
    )

data = [trace1]

layout = go.Layout(
    title_text='Queries that scanned the most percent from cache',
    yaxis=dict(
        # range = [0, 100],
        side = 'left',
        title="Percent Scanned From Cache"
        
    ),
    xaxis=dict(
        title="Query ID"

    )
)
fig = go.Figure(data=data, layout=layout)
fig.show()

#### Query 1

In [ ]:
print(df.iloc[2]["query_text"])

#### Query 2

In [ ]:
print(df.iloc[1]["query_text"])

#### Query 3

In [ ]:
print(df.iloc[2]["query_text"])

### Recommendations:

##TODO: @saravana to word recommendations appropriately and also recommend further analysis if any.

1) We need high reliability on most cached queries - since they are most used
2) Also same holds for most executed

## Most executed 'select' queries -- update this for select statements

In [ ]:
df=qqlib.n_most_executed_select_queries(sdate,edate,10)
# df.to_csv("/home/manas/DS_data/most_executed_select.csv")
df.head() 

##TODO: @saravana to put query filter rules so that this particular section surfaces the right reelvant queries for most executed

In [ ]:
## Queries that scanned the most from cache

trace1  = go.Scatter(
        mode='lines+markers',
        x = df.index,
        y = df['number_of_times_executed'],
        name="Number of times executed",
        marker_color='crimson'
    )

data = [trace1]

layout = go.Layout(
    title_text='Queries that were executed the most number of times',
    yaxis=dict(
        # range = [0, 100],
        side = 'left',
        title="Number of times executed"
        
    ),
    xaxis=dict(
        title="Query ID"

    )
)
fig = go.Figure(data=data, layout=layout)
fig.show()

#### Query 1

In [ ]:

print(df.iloc[0]["query_text"])

#### Query 2

In [ ]:
print(df.iloc[1]["query_text"])

#### Query 3

In [ ]:
print(df.iloc[3]["query_text"])

### ANALYSIS
* Query 1 was executed 116028 in 30 days
* Query 2 was executed 19770 in 30 days
* Query 3 was executed 13815 in 30 days.

### OPTIML Recommendations
* Can convert SELECT Queries as Materialized Views

## MOST EXECUTED QUERIES


In [ ]:
df=qqlib.caching_warehouse(sdate,edate,5)
df.head()

## LONGEST RUNNING QUERIES

In [ ]:
df=qqlib.longest_running_queries(sdate,edate,5)
df.head()

#### QUERY 1

In [ ]:
print(df.iloc[0]["query_text"])

#### QUERY 2

In [ ]:
print(df.iloc[1]["query_text"])

#### QUERY 3

In [ ]:
print(df.iloc[2]["query_text"])

### ANALYSIS

##TODO: @Saravana make a recommendation on how we recommend them to use SCD type 2 architecture

* Queries are the same.
* Updating event query.
* Execution time for query is approximately 135 minutes.
* 65% of total partitions of table are scanned.
* No spillage onto local or remote storage.
* No Query overloading.


### OPTIML to do

* Need to run update query to see if performance tuning can be done.

### OPTIML Recommendations
* Cluster keys can be defined to reduce execution time. WHERE clause present in query can be utilized.
* Warehouse scaling up won't help with performance.


# Notes

* For most expensive queries i.e. consuming most credits steps are:
  * Look at the amount of data the queries are operating on to see if it passes the sniff test
  * Idenitfy causes of credit consumption e.g. using INSERT instead of COPY_TO